In [1]:
#libraries
import cv2
import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean

In [2]:
#read video file frame by frame
def read_video(video,skip_frames,resolution):
    vid = cv2.VideoCapture(video)
    frames=[]
    vid_length=0
    while(vid.isOpened()):
        # Capture frame-by-frame
        ret, frame = vid.read() # if ret is false, frame has no content
        # resize the video to a different resolution
        if ret:
            frame=cv2.resize(frame,resolution)
        # skip every "skip_frame"
        if vid_length%skip_frames==0:
            frames.append(frame) #add the individual frames to a list
        vid_length+=1 #increase the vid_length counter
        if not ret:
            # Display the resulting frame
#             cv2.imshow('frame',frame)
#             if cv2.waitKey(1) & 0xFF == ord('q'):
#                 break
#         else:
#             break
            break
    vid.release()
    cv2.destroyAllWindows()
    return frames

In [3]:
frames=read_video('/home/jacob/Downloads/IMG_2525.MOV',2,(640,360))
len(frames)

2139

Dominant Color Extractor

In [4]:
def dominant3(image,rgb_to_color):
    #bins dict for histograms
    bins={}
    rgb_list=[]
    
    #init the dict with zeros for every key
    for rgb in rgb_to_color:
        bins[rgb_to_color[rgb]]=0
    #map the values of the dict to a list
    for rgb in rgb_to_color:
        rgb_list.append(rgb)
    #flatten the image to 1d 
    img = image.reshape((image.shape[0] * image.shape[1], 3))     
    
    #nearest neighbour search
    for pixel in img:
        bin_aux=[]
        #get the euclidean distance between the colors and the current pixel
        for rgb in rgb_list:
            bin_aux.append(euclidean(pixel,rgb))
        # get the index of the color,which has the smallest distance, in rgb_list
        min_pos = np.argmin(bin_aux)
        #increment the respective color 
        bins[rgb_to_color[rgb_list[min_pos]]]+=1
    #return the count of all the colors
    return bins

In [5]:
#read the file with colors and there rgb-values into a dictionary
rgb_to_color = {}
with open("colors") as f:
    for line in f:
        #split lines at "::
        color, rgb = line.strip().split(':')
        #strip the rgb-string of the parenthesis, split it up a the commas,
        #cast them to int and put them into a tuples
        rgb_value=tuple(map(int,
                            (rgb.strip('(').strip(')').split(','))))
        rgb_to_color[rgb_value] = color
len(rgb_to_color)
#purple is median purple
#skin is caucasian 

45

In [6]:
bins=dominant3(frames[0],rgb_to_color)
print(bins)

{'darkred': 131, 'firebrick': 0, 'crimson': 0, 'red': 0, 'tomato': 0, 'salmon': 0, 'dark_orange': 0, 'gold,': 0, 'dark_khaki': 1344, 'yellow': 0, 'dark_olive_green': 74141, 'olive_drab': 3745, 'green_yellow': 0, 'dark_green': 506, 'aqua_marine': 255, 'steel_blue': 8007, 'sky_blue': 4551, 'dark_blue': 4599, 'blue': 164, 'royal_blue': 1136, 'purple': 30, 'violet': 2, 'deep_pink': 0, 'pink': 6, 'antique_white': 278, 'saddle_brown': 562, 'sandy_brown': 0, 'ivory': 544, 'dim_grey': 40634, 'grey': 4660, 'silver': 7699, 'light_grey': 1454, 'black': 57587, 'white': 15217, 'dark_cyan': 83, 'cyan': 124, 'green': 0, 'khaki': 24, 'golden_rod': 0, 'orange': 0, 'coral': 0, 'magenta': 0, 'wheat': 162, 'skin': 43, 'purple4': 2712}


3. histogram

In [7]:
sort=zip(list(bins.values()),list(bins.keys()))
sort=sorted(sort)
df=pd.DataFrame(sort,columns=['count','color'])
df.set_index('color',inplace=True)
df.tail()

,count
color,
steel_blue,8007
white,15217
dim_grey,40634
black,57587
dark_olive_green,74141


In [8]:
df.plot(kind='bar')